In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import jieba 
from gensim.models import Word2Vec
from gensim.corpora.dictionary import Dictionary
import multiprocessing

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout,Activation
from keras.models import model_from_yaml
%matplotlib inline

d:\program files\python36\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
# set parameters(word2vec)
vocab_dim = 200
maxlen = 100
n_iterations = 1  # ideally more..
n_exposures = 10
window_size = 7
cpu_count = multiprocessing.cpu_count()

In [3]:
data=pd.read_csv('data/train_first.csv')
#data=data.drop(['Id'],axis=1)
#data.head()
#划分x  y
X=data['Discuss']
Y=data['Score']

In [4]:
#加载停用词
def get_stopwords(path):
    return [line.strip() for line in open(path,'r',encoding='utf-8').readlines()]
#句子去停用词
def removestopwords(sentence):
        stopwords_list=get_stopwords('data/stopwords.txt')
        outstr=[]
        for word in sentence:
            if not word in stopwords_list:
                if word!='\n' and word!='\t':
                     outstr.append(word)
        return outstr

In [5]:
#分词 并去掉停用词
def cut(sentence):
    return removestopwords(jieba.cut(sentence))
#分词后的word
cabs=[cut(x) for x in X]
#cabs[0]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ZHANGG~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.111 seconds.
Prefix dict has been built succesfully.


In [6]:
#word2vec

#创建词语字典，并返回每个词语的索引，词向量，以及每个句子所对应的词语索引
def word2vec_train(combined):

    model = Word2Vec(size=vocab_dim,
                     min_count=n_exposures,
                     window=window_size,
                     workers=cpu_count,
                     iter=n_iterations)
    model.build_vocab(combined)
    model.train(combined,total_examples=model.corpus_count,epochs=model.iter)
    model.save('data/Word2vec.pkl')
    index_dict, word_vectors,combined = create_dictionaries(model=model,combined=combined)
    return   index_dict, word_vectors,combined


#创建词语字典，并返回每个词语的索引，词向量，以及每个句子所对应的词语索引
def create_dictionaries(model=None,
                        combined=None):
    ''' Function does are number of Jobs:
        1- Creates a word to index mapping
        2- Creates a word to vector mapping
        3- Transforms the Training and Testing Dictionaries

    '''
    if (combined is not None) and (model is not None):
        gensim_dict = Dictionary()
        gensim_dict.doc2bow(model.wv.vocab.keys(),
                            allow_update=True)
        w2indx = {v: k+1 for k, v in gensim_dict.items()}#所有频数超过10的词语的索引
        w2vec = {word: model[word] for word in w2indx.keys()}#所有频数超过10的词语的词向量

        def parse_dataset(combined):
            ''' Words become integers
            '''
            data=[]
            for sentence in combined:
                new_txt = []
                for word in sentence:
                    try:
                        new_txt.append(w2indx[word])
                    except:
                        new_txt.append(0)
                data.append(new_txt)
            return data
        combined=parse_dataset(combined)
        combined= sequence.pad_sequences(combined, maxlen=maxlen)#每个句子所含词语对应的索引，所以句子中含有频数小于10的词语，索引为0
        return w2indx, w2vec,combined
    else:
        print ('No data provided...')

In [7]:
#根据词向量重新填充x
index_dict, word_vectors,combined=word2vec_train(cabs)


d:\program files\python36\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
#搭建网络结构
input_x=tf.placeholder(shape=[None,None,None],dtype=tf.float32,name='input_x')
lable=tf.placeholder(shape=[None,None],dtype=tf.int32,name='target')
lr=tf.placeholder(dtype=tf.float32,name='learning_rate')
sequence_lengths=tf.placeholder(shape=[None],dtype=tf.int32,name='sequence_lengths')
keep_prob=tf.placeholder(dtype=tf.float32,name='keep_prob')
#super params
hidden_dim=128
batch_size=100
layer_num=2
#bilstm
# with tf.variable_scope('bi_lstm'):
#     #lstm层
#     lstm_fw_cell=rnn.BasicLSTMCell(hidden_dim,forget_bias=1.0,state_is_tuple=True)
#     lstm_bw_cell=rnn.BasicLSTMCell(hidden_dim,forget_bias=1.0,state_is_tuple=True)
#     #dropout
#     lstm_fw_cell=rnn.DropoutWrapper(cell=lstm_fw_cell,input_keep_prob=1.0,out_keep_prob=keep_prob)
#     lstm_bw_cell=rnn.DropoutWrapper(cell=lstm_bw_cell,input_keep_prob=1.0,out_keep_prob=keep_prob)
#     #多层lstm
#     cell_fw=rnn.MultiRNNCell([lstm_fw_cell]*layer_num, state_is_tuple=True)
#     cell_bw=rnn.MultiRNNCell([lstm_fw_cell]*layer_num, state_is_tuple=True)
#     #初始状态
#     initial_state_fw=cell_fw.zero_state(batch_size,tf.float32)
#     initial_state_bw=cell_bw.zero_state(batch_size,tf.float32)

#lstm
with tf.variable_scope('lstm'):
    lstm_cell=tf.contrib.rnn.BasicLSTMCell(hidden_dim)
    def get_lstm_dropout():
        return tf.contrib.rnn.DropoutWrapper(cell=lstm_cell,output_keep_prob=keep_prob)
    cell=tf.contrib.rnn.MultiRNNCell([get_lstm_dropout()] for _ in range(layer_num))
    initial_state=cell.zero_state(batch_size,tf.float32)
    outputs, final_state=tf.nn.dynamic_rnn(cell,input_x,sequence_length=sequence_lengths,initial_state=initial_state)

In [ ]:
#训练

In [ ]:
#预测